In [ ]:
import (sys, os, re)
sys.path.append('..')  # To locate camelid

import pandas as pd
from pandas import DataFrame
from sqlalchemy import create_engine, types
from sqlalchemy.sql import text

from camelid.run import CamelidEnv
# TODO:
# from camelid.ids import cas_to_cid

In [ ]:
env = CamelidEnv(project='syn1701')
conn = create_engine('postgresql://akokai@localhost/cmlbase')

In [ ]:
# !psql cmlbase -c 'drop table unres2016;'

In [ ]:
cmd = text('''
create table unres2016 (casrn text, cid text, name text);
copy unres2016 from '/opt/akokai/data/camelid/syn1701/data/unresolved.tsv'
    with (format text, delimiter '\t', header);
''')
res = conn.execute(cmd)
print(res.rowcount)